In [2]:
import pandas as pd
import os

def average_court_time(player_name):
    file_path = f"/Users/eshaan/Desktop/Tennis Scouting/consulting-spring2025/data/mens/{player_name}/combined.xlsx"
    
    
    valid_games = []
    invalid_games = []

    try:
        xls = pd.ExcelFile(file_path)
        if 'Sets' in xls.sheet_names:
            df = xls.parse('Sets')

            if 'Duration' in df.columns and '__source_file__' in df.columns:
                df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')
                df = df[df['Duration'] < 10000]

                grouped = df.groupby('__source_file__')['Duration'].sum().reset_index()
                grouped['Court Time (minutes)'] = (grouped['Duration'] / 60).round(1)

                for _, row in grouped.iterrows():
                    match_name = os.path.splitext(row['__source_file__'])[0].replace('_', ' ')
                    minutes = row['Court Time (minutes)']
                    match_info = {
                        'Match': f"{player_name} vs {match_name}",
                        'Court Time (minutes)': minutes
                    }

                    if 40 <= minutes <= 120:
                        valid_games.append(match_info)
                    else:
                        invalid_games.append(match_info)

    except Exception as e:
        print(f"Error processing file for {player_name}: {e}")
        return None

    valid_df = pd.DataFrame(valid_games).sort_values(by='Court Time (minutes)', ascending=False).reset_index(drop=True)
    invalid_df = pd.DataFrame(invalid_games).sort_values(by='Court Time (minutes)', ascending=False).reset_index(drop=True)

    print("\n✅ Valid Match Durations (40–120 minutes):\n")
    print(valid_df.to_string(index=False))

    print("\n⚠️ Flagged Match Durations (outside 40–120 minutes):\n")
    print(invalid_df.to_string(index=False))

    if not valid_df.empty:
        average_minutes = round(valid_df['Court Time (minutes)'].mean(), 1)
        print(f"\nAverage court time for valid matches: {average_minutes} minutes")
        return average_minutes
    else:
        print("\nNo valid matches found.")
        return None
average_court_time("Rudy Quan")


✅ Valid Match Durations (40–120 minutes):

                                      Match  Court Time (minutes)
   Rudy Quan vs RudyQuan Pepperdine 2 27 25                 107.6
    Rudy Quan vs RudyQuan UMichigan 4 25 25                  86.0
          Rudy Quan vs RudyQuan USC 2 22 25                  79.3
 Rudy Quan vs RudyQuan DeaconThomas Indiana                  74.3
       Rudy Quan vs RudyQuan Indiana 3 7 25                  74.3
 Rudy Quan vs RudyQuan FloridaState 9 23 24                  66.6
Rudy Quan vs RudyQuan MichiganState 4 13 25                  57.2

⚠️ Flagged Match Durations (outside 40–120 minutes):

                             Match  Court Time (minutes)
 Rudy Quan vs RudyQuan Cal 1 25 25                   0.9
Rudy Quan vs RudyQuan TAMU 1 25 25                   0.4

Average court time for valid matches: 77.9 minutes


np.float64(77.9)